<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
        <img src="../figs/uoc_masterbrand_3linies_positiu.png", align="left">
    </div>
    <div style="float: right; width: 50%;">
        <p style="margin: 0; padding-top: 22px; text-align:right;">M2.855 · Modelos avanzados de minería de datos</p>
        <p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
        <p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
    </div>
</div>
<div style="width:100%;">&nbsp;</div>

# Ejemplo de combinación de clasificadores

En este ejemplo compararemos el uso de árboles de decisión y modelos _ensemble_, en concreto, el modelo **Random Forest**.


## Introducción

El _ensemble learning_ es una estrategia en la que se utiliza un grupo de modelos para resolver un problema mediante la combinación estratégica de varios modelos de aprendizaje automático en un solo modelo predictivo. 

En general, los métodos de ensemble se utilizan principalmente para mejorar la precisión del rendimiento general de un modelo y para combinar varios modelos diferentes, también conocidos como *aprendices básicos*, para predecir los resultados, en lugar de utilizar un solo modelo.

¿Por qué entrenamos tantos clasificadores diferentes en lugar de uno solo? El uso de varios modelos para predecir el resultado final en realidad reduce la probabilidad de sopesar las decisiones tomadas por modelos deficientes (sobreentrenados, no debidamente ajustados, etc.).

Cuanto más diversos sean estos aprendices básicos, más poderoso será el modelo final. 

Tengamos en cuenta que en cualquier modelo de aprendizaje automático, el error de generalización viene dado por la suma de cuadrados de bias + varianza + error irreductible. 

¡Los errores irreductibles son algo que está más allá de nosotros! No podemos reducirlos. 

Sin embargo, utilizando ensembles podemos reducir el sesgo (bias) y la varianza de un modelo. Esto reduce el error de generalización general.

La **compensación de sesgo-varianza** es el punto de referencia más importante que diferencia un modelo robusto de uno inferior (entendamos por inferior un modelo no demasiado generalizable). 

Aunque no es una regla exacta, en el aprendizaje automático, los modelos que tienen un sesgo alto tienden a tener una varianza más baja y viceversa.

Hemos estado hablando de bias y varianza. Pero veamos qué entendemos por sesgo de un modelo y por varianza de un modelo. 

1. **Sesgo**: el sesgo es un error que surge debido a suposiciones falsas realizadas en la fase de aprendizaje de un modelo. Un sesgo alto puede hacer que un algoritmo de aprendizaje omita información importante y correlaciones entre las variables independientes y las etiquetas de clase, por lo que no se ajusta al modelo.

2. **Varianza**: la varianza nos dice qué tan sensible es un modelo a los pequeños cambios en los datos de entrenamiento. Es decir, cuánto cambia el modelo. Una gran variación en un modelo lo hará propenso al ruido aleatorio presente en el conjunto de datos, por lo que se ajustará demasiado al modelo.

Para comprender con más detalle la compensación de sesgo y varianza en los modelos de aprendizaje automático, podéis consultar este [artículo](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229). 

Una vez llegados a este punto, podemos dividir los ensembles en cuatro categorías: 

1. **Bagging**: el bagging se utiliza principalmente para reducir la variación en un modelo. Un ejemplo simple de bagging es el algoritmo Random Forest.

2. **Boosting**: el boosting se utiliza principalmente para reducir el sesgo en un modelo. Ejemplos de algoritmos de impulso son Ada-Boost, XGBoost, árboles de decisión mejorados por gradiente, etc.

3. **Stacking**: el stacking se utiliza principalmente para aumentar la precisión de predicción de un modelo. Para implementar el stacking usaremos la biblioteca mlextend proporcionada por scikit learn.

4. **Cascading**: esta clase de modelos son muy precisos. La conexión en cascada se usa principalmente en escenarios en los que no puede permitirse cometer un error. Por ejemplo, una técnica en cascada se usa principalmente para detectar transacciones fraudulentas con tarjetas de crédito.

## Datos

Para este ejercicio usaremos el dataset *diabetes.csv*. Este conjunto de datos es original del Instituto Nacional de Diabetes y Enfermedades Digestivas y Renales. El objetivo de este dataset es predecir, basándonos en las mediciones de diagnóstico, si un paciente tiene diabetes.

En particular, todos los pacientes aquí son mujeres de al menos 21 años, de ascendencia india Pima.

El dataset contiene la siguiente información

- Embarazos: número de embarazos
- Glucosa: concentración de glucosa en plasma a 2 horas en una prueba de tolerancia a la glucosa oral
- Presión arterial: presión arterial diastólica (mm Hg)
- SkinThickness: espesor del pliegue cutáneo del tríceps (mm)
- Insulina: insulina sérica de 2 horas (mu U / ml)
- IMC: índice de masa corporal (peso en kg / (altura en m) ^ 2)
- DiabetesPedigreeFunction: función del pedigrí de la diabetes
- Edad: edad (años)
- Resultado (variable objetivo): variable de clase (0 o 1) 

En la primera parte de este ejemplo veremos la combinación de clasificadores en paralelo mediante las técnicas de **_Bagging_** y **_Boosting_**.

Para empezar, veamos cómo es el dataset.

In [23]:
diabetes = pd.read_csv('../data/diabetes.csv')

nRow, nCol = diabetes.shape
print(f'Hay {nRow} filas y {nCol} columnas')
diabetes.tail()

Hay 768 filas y 9 columnas


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


Para poder probar varios modelos, primero vamos a dividir el dataset entre _train_ y _test_.

Para que todos obtengáis los mismos resultados y poder comentar dudas por el foro, fijaremos la seed para obtener los mismos datasets de train y test.

Como en este ejercicio trataremos *stacking* y *cascading*, y ambos se aplican sobre el conjunto de test, haremos un *split* del 60 % para tener un poco más de base al aplicar estas dos técnicas.

In [24]:
myseed= 38
X = diabetes.drop(columns = 'Outcome')
y = diabetes['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=myseed)

## Combinación paralela de clasificadores

### 1. Árboles de decisión

Para poder comparar el aumento de  *performance* obtenido a medida que vamos aplicando técnicas nuevas, utilizaremos como  *baseline* un simple árbol de decisión.

A continuación, entrenaremos un árbol de decisión sobre el conjunto de datos de train con profundidad máxima de 3 niveles (aplicaremos la misma restricción en las siguientes secciones), y evaluaremos sobre test y calcularemos su precisión aplicando validación cruzada con 5 conjuntos.
   
<u>Más información</u>: 
- [*cross_val_score*](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
- [*cross validation*](http://scikit-learn.org/stable/modules/cross_validation.html)

In [25]:
clf = DecisionTreeClassifier(max_depth=3, random_state=myseed)

cvscores = cross_val_score(clf, X_train, y_train, cv=5)
print("Precisión media obtenida con CV: {:.2f} +/- {:.2f} %".format(np.mean(cvscores)*100, np.std(cvscores)*100))

Precisión media obtenida con CV: 71.04 +/- 4.82 %


In [26]:
clf.fit(X_train, y_train)
y_pred_tree = clf.predict(X_test)
accuracy_score(y_pred_tree, y_test)

0.7462039045553145

Obtenemos una precisión en test no demasiado alta, un poco por debajo de la de train (esto puede variar dependiendo de los datasets de train y test). 

### 2. *Bagging*

La idea central del bagging es usar réplicas del conjunto de datos original y usarlas para entrenar diferentes clasificadores.

Crearemos subconjuntos muestreando aleatoriamente un montón de puntos del conjunto de datos de entrenamiento con reemplazo. 

Ahora entrenaremos clasificadores individuales en cada uno de estos subconjuntos bootstrap. 

Cada uno de estos clasificadores base predecirá la etiqueta de clase para un problema dado. Aquí es donde combinamos las predicciones de todos los modelos base. Esta parte se llama etapa de agregación. Es por eso que encontraréis los ensembles bagging por el nombre de ensembles de agregación. 

Por lo general, se usa un voto de mayoría simple en un sistema de clasificación y se toma la media de todas las predicciones para los modelos de regresión para combinar todos los clasificadores base en un solo modelo y proporcionar el resultado final del modelo de conjunto. 

Un ejemplo simple de tal enfoque es el algoritmo Random Forest. El bagging reduce la alta variación (varianza) de un modelo, reduciendo así el error de generalización. Es un método muy eficaz, especialmente cuando tenemos datos muy limitados como pudiera ser nuestro caso. 

Mediante el uso de muestras de bootstrap, podemos obtener una estimación agregando las puntuaciones de muchas muestras.

**¿Cómo haríamos el bagging?**

Supongamos que tenemos un conjunto de entrenamiento que contiene 100.000 puntos de datos. 

Crearíamos N subconjuntos muestreando al azar 50K puntos de datos para cada subconjunto. 

Cada uno de estos N subconjuntos se utilizará para entrenar N clasificadores diferentes. 

En la etapa de agregación, todas estas N predicciones se combinarán en un solo modelo, también llamado metaclasificador. 

De los 100.000 puntos presentes originalmente en el conjunto de datos, si eliminamos 1000 puntos, el impacto que tendrá en los conjuntos de datos muestreados será muy inferior. 

Si pensamos intuitivamente, es posible que algunos de estos 1000 puntos no estén presentes en todos los conjuntos de datos muestreados y, por lo tanto, la cantidad de puntos que se eliminarán de cada conjunto de datos muestreados será muy inferior. ¡Incluso cero en algunos casos! En resumen, el impacto de eliminar 1000 puntos de este tipo será en realidad menor en los clasificadores base, lo que reducirá la variación en un modelo y lo hará más sólido. 

La varianza no es más que sensibilidad al ruido, como hemos comentado anteriormente.

Seguidamente, entrenaremos un _**Random Forest**_ sobre el conjunto de datos de _train_ con <b>20 árboles</b> de decisión y <b>profundidad máxima de 3</b> niveles, y evaluaremos sobre test y calcularemos su precisión aplicando validación cruzada con 5 conjuntos.

<u>Más información</u>: 
- [*RandomForestClassifier*](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [27]:
clf = ensemble.RandomForestClassifier(n_estimators=20, max_depth=3, random_state=myseed)

cvscores = cross_val_score(clf, X_train, y_train, cv=5)
print("Precisión media obtenida con CV: {:.2f} +/- {:.2f} %".format(np.mean(cvscores)*100, np.std(cvscores)*100))

Precisión media obtenida con CV: 73.29 +/- 1.57 %


In [28]:
clf.fit(X_train, y_train)
y_pred_random_forest = clf.predict(X_test)
accuracy_score(y_pred_random_forest, y_test)

0.7418655097613883

Obtenemos una buena precisión en test, un poco por debajo de la de train. También notamos una mejora (aunque leve) respecto a un simple árbol de decisión.

### 3. *Boosting*

El _boosting_ se utiliza para convertir a los clasificadores de base débil en fuertes. Los clasificadores débiles generalmente tienen una correlación muy débil con las etiquetas de clase verdaderas y los clasificadores fuertes tienen una correlación muy alta entre el modelo y las etiquetas de clase verdaderas.

El _boosting_ capacita a los clasificadores débiles de manera iterativa, cada uno tratando de corregir el error cometido por el modelo anterior. Esto se logra entrenando un modelo débil en todos los datos de entrenamiento, luego construyendo un segundo modelo que tiene como objetivo corregir los errores cometidos por el primer modelo. Luego construimos un tercer modelo que intenta corregir los errores cometidos por el segundo modelo y así sucesivamente. Los modelos se agregan de forma iterativa hasta que el modelo final ha corregido todos los errores cometidos por todos los modelos anteriores.

Cuando se agregan los modelos en cada etapa, se asignan algunos pesos al modelo que está relacionado con la precisión del modelo anterior. Después de agregar un clasificador débil, los pesos se vuelven a ajustar. Los puntos clasificados incorrectamente reciben pesos más altos y los puntos clasificados correctamente reciben pesos más bajos. Este enfoque hará que el siguiente clasificador se centre en los errores cometidos por el modelo anterior.

El boosting reduce el error de generalización tomando un modelo de alto bias y baja varianza y reduciendo el bias en un nivel significativo. Recuerda, el bagging reduce la varianza. Al igual que el bagging, el boosting también nos permite trabajar con modelos de clasificación y regresión. 

Entrenaremos un _**Gradient Boosting**_ sobre el conjunto de datos de _train_ con 20 árboles de decisión y profundidad máxima de 3 niveles. A continuación, evaluaremos sobre test y calcularemos su precisión aplicando validación cruzada con 5 conjuntos.

<u>Más información</u>: 
- [*GradientBoostingClassifier*](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

In [37]:
clf = ensemble.GradientBoostingClassifier(n_estimators=20, max_depth=3, random_state=myseed)

cvscores = cross_val_score(clf, X_train, y_train, cv=5)
print("Precisión media obtenida con CV: {:.2f} +/- {:.2f} %".format(np.mean(cvscores)*100, np.std(cvscores)*100))

Precisión media obtenida con CV: 72.00 +/- 4.23 %


In [30]:
clf.fit(X_train, y_train)
y_pred_gradient_boosting = clf.predict(X_test)
accuracy_score(y_pred_gradient_boosting, y_test)

0.7635574837310195